In [ ]:
import torch
import numpy as np
from random import seed

from vulture.main import get_dv2_model, get_upsampler_and_expr

from interactive_seg_backend.configs import FeatureConfig, TrainingConfig
from is_helpers import train_model_over_images, apply_model_over_images, eval_preds

from typing import Literal

SEED = 10672
np.random.seed(SEED)
torch.manual_seed(SEED)
seed(SEED)
DEVICE = "cuda:1"

N CPUS: 110


In [2]:
dv2 = get_dv2_model(True, device=DEVICE)

model_path = "../trained_models/e5000_full_fit_reg.pth"
cfg_path = "../vulture/models/configs/combined_no_shift.json"

upsampler, expr = get_upsampler_and_expr(model_path, cfg_path, device=DEVICE)

Using cache found in /home/ronan/.cache/torch/hub/ywyue_FiT3D_main


[128, 128, 128, 128]


In [3]:
PATH = "fig_data/is_benchmark"
AllowedDatasets = Literal["Ni_superalloy_SEM"]
dataset: tuple[AllowedDatasets, ...] = ("Ni_superalloy_SEM", )

TRAIN_IMG_FNAMES: dict[AllowedDatasets, list[str]] = {
                                                      "Ni_superalloy_SEM": ["000", "001", "005", "007"], 
                                                      }

all_classical_preds: dict[AllowedDatasets, dict[str, np.ndarray]] = {k: {} for k in dataset}


In [4]:
k_truncates = (1, 16, 32, 64, 128)
all_deep_preds: dict[int, dict[str, np.ndarray]] = {k: {} for k in k_truncates}

for k in k_truncates:
    SEED = 10672
    seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    chosen_dataset = 'Ni_superalloy_SEM'
    feat_cfg = FeatureConfig()

    deep_train_cfg = TrainingConfig(feat_cfg, n_samples=-1, add_dino_features=True, classifier='xgb', classifier_params = {"class_weight": "balanced", "max_depth": 32},)
    deep_model, pca = train_model_over_images(chosen_dataset, deep_train_cfg, PATH, TRAIN_IMG_FNAMES[chosen_dataset], dv2, upsampler, expr, K=k, overwrite_with_gt=True)

    deep_preds = apply_model_over_images(chosen_dataset, deep_train_cfg, deep_model, PATH, dv2, upsampler, expr, K=k, verbose=True, existing_pca=pca)

    all_deep_preds[k] = deep_preds

Finished featurising
Finished featurising
Finished featurising
Finished featurising
[00/23] - 000.tif
[10/23] - 010.tif
[20/23] - 020.tif
Finished featurising
Finished featurising
Finished featurising
Finished featurising
[00/23] - 000.tif
[10/23] - 010.tif
[20/23] - 020.tif
Finished featurising
Finished featurising
Finished featurising
Finished featurising
[00/23] - 000.tif
[10/23] - 010.tif
[20/23] - 020.tif
Finished featurising
Finished featurising
Finished featurising
Finished featurising
[00/23] - 000.tif
[10/23] - 010.tif
[20/23] - 020.tif
Finished featurising
Finished featurising
Finished featurising
Finished featurising
[00/23] - 000.tif
[10/23] - 010.tif
[20/23] - 020.tif


In [5]:
for k in  k_truncates:   
    deep_preds = all_deep_preds[k]
    miou_deep, miou_std_deep = eval_preds(chosen_dataset, deep_preds, PATH )
    print(f"======== {chosen_dataset} ========")
    print(f"{k}: {miou_deep:.4f}+/-{miou_std_deep:.4f}\n")

======== Ni_superalloy_SEM ========
1: 0.5905+/-0.1527

======== Ni_superalloy_SEM ========
16: 0.7216+/-0.1588

======== Ni_superalloy_SEM ========
32: 0.7273+/-0.1589

======== Ni_superalloy_SEM ========
64: 0.7350+/-0.1593

======== Ni_superalloy_SEM ========
128: 0.7424+/-0.1562

